# Blood-Brain Barrier Permeability Prediction using Artificial Neural Networks

This notebook implements an ANN model for predicting blood-brain barrier (BBB) permeability with:
- **Hyperparameter Tuning**: Number of layers, neurons per layer, learning rate, batch size
- **Wandb Integration**: For visualisation of results and comparison of different models
- **Feature Engineering**: Molecular descriptors + MACCS fingerprints as input to model
- **Dataset**: B3DB dataset

## Table of Contents
1. Import Libraries
2. Load and Explore Data
3. Feature Extraction
4. Data Preprocessing
5. ANN Model Architecture
6. Hyperparameter Tuning with Wandb
7. Model Training
8. Evaluation and Results

## 1. Import Required Libraries

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
import warnings
warnings.filterwarnings('ignore')

from rdkit import Chem
from rdkit.Chem import Descriptors, Lipinski, Crippen, MACCSkeys

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             classification_report, confusion_matrix, roc_curve, auc,
                             roc_auc_score, matthews_corrcoef)

# Wandb for experiment tracking
import wandb

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


## 2. Load and Explore Data

In [3]:
data_path = Path('../data/B3DB_classification.tsv')
df = pd.read_csv(data_path, sep='\t')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
print(df.head())

print(f"\nDataset info:")
print(df.info())

print(f"\nTarget distribution:")
print(df['BBB+/BBB-'].value_counts())
print(f"\nClass balance:")
print(df['BBB+/BBB-'].value_counts(normalize=True))

print(f"\nMissing values:")
print(df.isnull().sum())

Dataset shape: (7807, 12)

First few rows:
   NO.                        compound_name  \
0    1                       sulphasalazine   
1    2                           moxalactam   
2    3                           clioquinol   
3    4  bbcpd11 (cimetidine analog) (y-g13)   
4    5                        schembl614298   

                                          IUPAC_name  \
0  2-hydroxy-5-[[4-(pyridin-2-ylsulfamoyl)phenyl]...   
1  7-[[2-carboxy-2-(4-hydroxyphenyl)acetyl]amino]...   
2                       5-chloro-7-iodoquinolin-8-ol   
3  2-[2-[(3-bromopyridin-2-yl)methylsulfanyl]ethy...   
4  (2s,3s,4s,5r)-6-[[(4r,4ar,7s,7ar,12bs)-7-hydro...   

                                              SMILES         CID  logBB  \
0   O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O      5339.0  -2.69   
1  COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...      3889.0  -2.52   
2                             Oc1c(I)cc(Cl)c2cccnc12      2788.0  -2.40   
3                         CCNC(=NCCSC

## 3. Feature Extraction

We'll extract two types of features:
1. **Molecular Descriptors**: Physical and chemical properties (12 descriptors)
2. **MACCS Fingerprints**: 167-bit structural fingerprints

Total feature dimension: 12 + 167 = 179 features

In [4]:
def extract_molecular_descriptors(smiles):
    """Extract molecular descriptors from SMILES string"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    descriptors = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Crippen.MolLogP(mol),
        'NumHDonors': Lipinski.NumHDonors(mol),
        'NumHAcceptors': Lipinski.NumHAcceptors(mol),
        'TPSA': Descriptors.TPSA(mol),
        'NumRotatableBonds': Lipinski.NumRotatableBonds(mol),
        'NumAromaticRings': Lipinski.NumAromaticRings(mol),
        'NumHeteroatoms': Lipinski.NumHeteroatoms(mol),
        'NumRings': Lipinski.RingCount(mol),
        'FractionCsp3': Lipinski.FractionCSP3(mol),
        'NumSaturatedRings': Lipinski.NumSaturatedRings(mol),
        'NumAliphaticRings': Lipinski.NumAliphaticRings(mol),
    }
    return descriptors

def extract_maccs_fingerprint(smiles):
    """Extract MACCS fingerprint from SMILES string"""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    maccs = MACCSkeys.GenMACCSKeys(mol)
    return np.array(maccs)

def extract_features(smiles):
    """Extract combined features: molecular descriptors + MACCS fingerprints"""
    descriptors = extract_molecular_descriptors(smiles)
    if descriptors is None:
        return None
    
    maccs = extract_maccs_fingerprint(smiles)
    if maccs is None:
        return None
    
    # Combine features
    descriptor_values = list(descriptors.values())
    combined_features = np.concatenate([descriptor_values, maccs])
    
    return combined_features


In [18]:
features_list = []
valid_indices = []

for idx, smiles in enumerate(df['SMILES']):
    features = extract_features(smiles)
    if features is not None:
        features_list.append(features)
        valid_indices.append(idx)
    
    if (idx + 1) % 500 == 0:
        print(f"Processed {idx + 1}/{len(df)} molecules...")

X = np.array(features_list)
y_series = pd.Series(df.loc[valid_indices, 'BBB+/BBB-'])
y = y_series.replace({'BBB+': 1, 'BBB-': 0}).astype(np.float32).values

print(f"Total samples: {len(X)}")
print(f"Feature dimension: {X.shape[1]}")
print(f"Valid molecules: {len(valid_indices)}/{len(df)}")
print(f"\nFeature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")

Processed 500/7807 molecules...
Processed 1000/7807 molecules...
Processed 1500/7807 molecules...
Processed 2000/7807 molecules...
Processed 2500/7807 molecules...
Processed 3000/7807 molecules...
Processed 3500/7807 molecules...
Processed 4000/7807 molecules...
Processed 4500/7807 molecules...
Processed 5000/7807 molecules...


[20:47:02] Explicit valence for atom # 10 C, 4, is greater than permitted


Processed 5500/7807 molecules...
Processed 6000/7807 molecules...
Processed 6500/7807 molecules...
Processed 7000/7807 molecules...
Processed 7500/7807 molecules...
Total samples: 7805
Feature dimension: 179
Valid molecules: 7805/7807

Feature matrix shape: (7805, 179)
Target vector shape: (7805,)


[20:47:08] Explicit valence for atom # 10 C, 4, is greater than permitted


## 4. Data Preprocessing

Split the data into training and test sets, then standardize features. [We use stratified sampling]

In [19]:
# Split data into train and test sets (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")
print(f"\nTraining set class distribution:")
print(pd.Series(y_train).value_counts(normalize=True))
print(f"\nTest set class distribution:")
print(pd.Series(y_test).value_counts(normalize=True))

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Scaled training data shape: {X_train_scaled.shape}")
print(f"Scaled test data shape: {X_test_scaled.shape}")

Training set size: 6244
Test set size: 1561

Training set class distribution:
1.0    0.63501
0.0    0.36499
Name: proportion, dtype: float64

Test set class distribution:
1.0    0.634849
0.0    0.365151
Name: proportion, dtype: float64
Scaled training data shape: (6244, 179)
Scaled test data shape: (1561, 179)


## 5. ANN Model Architecture

Define a flexible neural network architecture that supports variable number of layers and neurons.

In [7]:
class BBBPredictorANN(nn.Module):
    """
    ANN architecture for BBB permeability prediction
    
    Parameters:
    -----------
    input_size : int
        Dimension of input features
    hidden_layers : list of int
        Number of neurons in each hidden layer
    dropout_rate : float
        Dropout rate for regularization
    """
    def __init__(self, input_size, hidden_layers, dropout_rate=0.3):
        super(BBBPredictorANN, self).__init__()
        
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_size))
            layers.append(nn.Dropout(dropout_rate))
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, 1))
        layers.append(nn.Sigmoid())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    """Train the ANN model and track performance"""
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []
    
    best_val_acc = 0.0
    best_model_state = None
    
    for epoch in range(num_epochs):
        #training
        start_time = time.time()
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            # Forward pass
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y.unsqueeze(1))
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Calculate accuracy
            predictions = (outputs > 0.5).float()
            train_correct += (predictions == batch_y.unsqueeze(1)).sum().item()
            train_total += batch_y.size(0)
            train_loss += loss.item()
        
        avg_train_loss = train_loss / len(train_loader)
        train_acc = train_correct / train_total
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y.unsqueeze(1))
                
                predictions = (outputs > 0.5).float()
                val_correct += (predictions == batch_y.unsqueeze(1)).sum().item()
                val_total += batch_y.size(0)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        val_acc = val_correct / val_total if val_total > 0 else 0
        
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)
        
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'train_accuracy': train_acc,
            'val_accuracy': val_acc
        })
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict().copy()
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}] - "
                  f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} - "
                  f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, "
                  f"Time for epoch: {time.time() - start_time:.2f} seconds")
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accs': train_accs,
        'val_accs': val_accs,
        'best_val_acc': best_val_acc
    }

def evaluate_model(model, test_loader, device):
    """Evaluate the model on test set"""
    model.eval()
    all_predictions = []
    all_labels = []
    all_probs = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            outputs = model(batch_X)
            predictions = (outputs > 0.5).float()
            
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
            all_probs.extend(outputs.cpu().numpy())
    
    all_predictions = np.array(all_predictions).flatten()
    all_labels = np.array(all_labels).flatten()
    all_probs = np.array(all_probs).flatten()
    
    accuracy = accuracy_score(all_labels, all_predictions)
    precision = precision_score(all_labels, all_predictions)
    recall = recall_score(all_labels, all_predictions)
    f1 = f1_score(all_labels, all_predictions)
    roc_auc = roc_auc_score(all_labels, all_probs)
    mcc = matthews_corrcoef(all_labels, all_predictions)
    
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'mcc': mcc
    }
    
    return metrics, all_predictions, all_labels, all_probs


## 6. Hyperparameter Tuning with Wandb

We'll tune the following hyperparameters:
- **Number of hidden layers**: 2, 3, 4
- **Neurons per layer**: 64, 128, 256
- **Learning rate**: 0.0001, 0.001, 0.01
- **Batch size**: 16, 32, 64
- **Dropout rate**: 0.2, 0.3, 0.4

In [14]:
# WandB sweep configuration (Bayesian optimization, optimize AUC)
sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'test_roc_auc', 'goal': 'maximize'},
    'early_terminate': {
        'type': 'hyperband',
        'min_iter': 10
    },
    'parameters': {
        'num_layers': {'values': [2, 3, 4]},
        'neurons': {'values': [128, 256,512]},
        'learning_rate': {'distribution': 'log_uniform_values', 'min': 1e-4, 'max': 1e-2},
        'batch_size': {'values': [16, 32, 64]},
        'dropout': {'values': [0.2, 0.3, 0.4]},
        'num_epochs': {'value': 100}
    }
}

In [12]:
def run_experiment(config, X_train, y_train, X_test, y_test, input_size, num_epochs=100, init_wandb=True):
    """
    Run a single experiment with given hyperparameters
    
    Parameters:
    -----------
    config : dict
        Hyperparameter configuration
    X_train, y_train : numpy arrays
        Training data
    X_test, y_test : numpy arrays
        Test data
    input_size : int
        Input feature dimension
    num_epochs : int
        Number of training epochs
    
    Returns:
    --------
    metrics : dict
        Test set performance metrics
    """
    # Initialize wandb run (or reuse existing run when called from a sweep)
    if init_wandb:
        run = wandb.init(
            project="bbb-permeability-ann",
            config=config,
            reinit=True
        )
    else:
        run = wandb.run
    
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
    )
    
    train_dataset = TensorDataset(
        torch.FloatTensor(X_tr),
        torch.FloatTensor(y_tr)
    )
    val_dataset = TensorDataset(
        torch.FloatTensor(X_val),
        torch.FloatTensor(y_val)
    )
    test_dataset = TensorDataset(
        torch.FloatTensor(X_test),
        torch.FloatTensor(y_test)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)
    
    hidden_layers = [config['neurons']] * config['num_layers']
    model = BBBPredictorANN(
        input_size=input_size,
        hidden_layers=hidden_layers,
        dropout_rate=config['dropout']
    ).to(device)
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
    
    print(f"\nTraining with config: {config}")
    model, history = train_model(
        model, train_loader, val_loader, criterion, optimizer, num_epochs, device
    )
    
    metrics, predictions, labels, probs = evaluate_model(model, test_loader, device)
    
    wandb.log({
        'test_accuracy': metrics['accuracy'],
        'test_precision': metrics['precision'],
        'test_recall': metrics['recall'],
        'test_f1_score': metrics['f1_score'],
        'test_roc_auc': metrics['roc_auc'],
        'test_mcc': metrics['mcc']
    })
    
    cm = confusion_matrix(labels, predictions)
    wandb.log({
        "confusion_matrix": wandb.plot.confusion_matrix(
            probs=None,
            y_true=labels.astype(int),
            preds=predictions.astype(int),
            class_names=["Non-permeable", "Permeable"]
        )
    })
    
    print(f"Test Accuracy: {metrics['accuracy']:.4f}")
    print(f"Test F1-Score: {metrics['f1_score']:.4f}")
    print(f"Test ROC-AUC: {metrics['roc_auc']:.4f}")
    
    if init_wandb:
        run.finish()
    
    return metrics, model, history

print("Experiment function defined successfully!")

Experiment function defined successfully!


## 7. Run Hyperparameter Tuning Experiments

Now we'll run all hyperparameter configurations and track results with Wandb.

In [20]:
input_size = X_train_scaled.shape[1]
num_epochs = 100

results = []
best_f1 = 0
best_config = None
best_model = None

print("Starting W&B sweep (Bayesian search) for hyperparameter tuning...")

sweep_id = wandb.sweep(sweep_config, project="bbb-permeability-ann")

def _sweep_run():
    with wandb.init(project="bbb-permeability-ann", reinit=True) as run:
        cfg = dict(run.config)
        metrics, model, history = run_experiment(
            config=cfg,
            X_train=X_train_scaled,
            y_train=y_train,
            X_test=X_test_scaled,
            y_test=y_test,
            input_size=input_size,
            num_epochs=cfg.get('num_epochs', 100),
            init_wandb=True
        )

        # Save run-specific results locally if desired
        results.append({**cfg, **metrics})

wandb.agent(sweep_id, function=_sweep_run, count=20)

Starting W&B sweep (Bayesian search) for hyperparameter tuning...
Create sweep with ID: y8uxcdqx
Sweep URL: https://wandb.ai/bhuvan-kapur1-iiith/bbb-permeability-ann/sweeps/y8uxcdqx


wandb: Agent Starting Run: f85eoc67 with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	learning_rate: 0.0014566801663309075
wandb: 	neurons: 128
wandb: 	num_epochs: 100
wandb: 	num_layers: 2


wandb: ERROR The nbformat package was not found. It is required to save notebook history.



Training with config: {'batch_size': 32, 'dropout': 0.3, 'learning_rate': 0.0014566801663309075, 'neurons': 128, 'num_epochs': 100, 'num_layers': 2}
Epoch [10/100] - Train Loss: 0.2512, Train Acc: 0.8927 - Val Loss: 0.2823, Val Acc: 0.8775, Time for epoch: 0.65 seconds
Epoch [20/100] - Train Loss: 0.2122, Train Acc: 0.9105 - Val Loss: 0.2986, Val Acc: 0.8823, Time for epoch: 0.54 seconds
Epoch [30/100] - Train Loss: 0.1784, Train Acc: 0.9315 - Val Loss: 0.3643, Val Acc: 0.8727, Time for epoch: 0.55 seconds
Epoch [40/100] - Train Loss: 0.1613, Train Acc: 0.9379 - Val Loss: 0.3441, Val Acc: 0.8743, Time for epoch: 0.52 seconds
Epoch [50/100] - Train Loss: 0.1430, Train Acc: 0.9415 - Val Loss: 0.3546, Val Acc: 0.8751, Time for epoch: 0.52 seconds
Epoch [60/100] - Train Loss: 0.1523, Train Acc: 0.9477 - Val Loss: 0.3871, Val Acc: 0.8799, Time for epoch: 0.56 seconds
Epoch [70/100] - Train Loss: 0.1228, Train Acc: 0.9516 - Val Loss: 0.4396, Val Acc: 0.8871, Time for epoch: 0.67 seconds
Epo

epoch,▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇████
test_accuracy,▁
test_f1_score,▁
test_mcc,▁
test_precision,▁
test_recall,▁
test_roc_auc,▁
train_accuracy,▁▃▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇██████████████
train_loss,█▇▆▆▅▅▄▄▄▄▃▃▂▃▂▂▂▂▂▂▃▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▄▅▅▆▆▆▆▆▆█▆▆▇▄▅▆▆▆▆▆▆▇█▆▇▆▆▇▇▆▇█▆▆▆▆▅▆
val_loss,▃▂▂▁▁▁▂▁▁▁▂▃▂▂▄▃▃▂▃▄▃▅▃▃▄▃▃▅▄▄▄▇███▄▃▆▅▄


...

wandb: Agent Starting Run: 0yu4gvlg with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.3
wandb: 	learning_rate: 0.006528972074775396
wandb: 	neurons: 512
wandb: 	num_epochs: 100
wandb: 	num_layers: 3


wandb: ERROR The nbformat package was not found. It is required to save notebook history.



Training with config: {'batch_size': 32, 'dropout': 0.3, 'learning_rate': 0.006528972074775396, 'neurons': 512, 'num_epochs': 100, 'num_layers': 3}
Epoch [10/100] - Train Loss: 0.2681, Train Acc: 0.8855 - Val Loss: 0.3037, Val Acc: 0.8751, Time for epoch: 0.92 seconds
Epoch [20/100] - Train Loss: 0.2058, Train Acc: 0.9191 - Val Loss: 0.3122, Val Acc: 0.8839, Time for epoch: 0.90 seconds
Epoch [30/100] - Train Loss: 0.1598, Train Acc: 0.9349 - Val Loss: 0.3305, Val Acc: 0.8791, Time for epoch: 0.79 seconds
Epoch [40/100] - Train Loss: 0.1566, Train Acc: 0.9371 - Val Loss: 0.4000, Val Acc: 0.8791, Time for epoch: 0.85 seconds
Epoch [50/100] - Train Loss: 0.1368, Train Acc: 0.9469 - Val Loss: 0.4363, Val Acc: 0.8735, Time for epoch: 0.82 seconds
Epoch [60/100] - Train Loss: 0.1493, Train Acc: 0.9483 - Val Loss: 0.8781, Val Acc: 0.8631, Time for epoch: 0.79 seconds
Epoch [70/100] - Train Loss: 0.1159, Train Acc: 0.9514 - Val Loss: 0.4742, Val Acc: 0.8815, Time for epoch: 0.93 seconds
Epoc

epoch,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇█████
test_accuracy,▁
test_f1_score,▁
test_mcc,▁
test_precision,▁
test_recall,▁
test_roc_auc,▁
train_accuracy,▁▂▄▅▅▆▆▇▆▇▇▇▇▇▇▇▇▇▇███▇█████████▇▇██████
train_loss,▇▆▅▅▅▄▄▃▃▃▂▂▃▃▃▂▂▃▂▂▂▂▁▁▂▁▂▂▂▂▂▂▁▁▁▁▁▁▂█
val_accuracy,▁▁▂▄▃▅▇▇▆▆▄▅▄▄▅▅▄▅▅▆▅▅▆▅▆▄▇▆▅▆▆▆▆▅▇▆▅▇█▇
val_loss,▁▁▁▁▂▂▁▁▂▂▁▁▁▂▂▂▃▂▃▂▃▃▂▄▃▅▄▆▇▄▇▄▃▆▇▆█▅▆█


W&B sweep launched. Use the W&B UI to monitor runs.


## 8. Results Analysis

Let's analyze the results from all experiments.

In [22]:
results_df = pd.DataFrame(results)

# Sort by ROC AUC
results_df_sorted = results_df.sort_values('roc_auc', ascending=False)

print("Top 5 configurations by ROC AUC:")
print(results_df_sorted.head())

# Save results to CSV
results_path = Path('../figures/B3DB_classification/')
results_path.mkdir(parents=True, exist_ok=True)
results_df_sorted.to_csv(results_path / 'ann_hyperparameter_results.csv', index=False)

Top 5 configurations by ROC AUC:
    batch_size  dropout  learning_rate  neurons  num_epochs  num_layers  \
14          32      0.4       0.000221      128         100           3   
12          64      0.4       0.001024      128         100           4   
16          64      0.4       0.001513      128         100           4   
5           64      0.4       0.001134      128         100           4   
15          64      0.3       0.000125      512         100           4   

    accuracy  precision    recall  f1_score   roc_auc       mcc  
14  0.898783   0.922843  0.917255  0.920040  0.955620  0.782200  
12  0.892377   0.900682  0.933401  0.916749  0.952697  0.765686  
16  0.893658   0.909633  0.924319  0.916917  0.952093  0.769447  
5   0.892377   0.905419  0.927346  0.916251  0.951051  0.766203  
15  0.885971   0.903674  0.918264  0.910911  0.950967  0.752762  


## Best ROC-AUC : 0.955620

### Training Configuration : 3 layers, 128 neurons in each layer, trained for 100 epochs, dropout rate = 0.4, batch size = 32